Sophia and Harry's Project!!!!!!!


Steps:

1) get input data
2) build model architecture
3) write the loss functions
4) write the training procedure


In [0]:
from google.colab import drive
drive.mount('/content/drive')

# New Section

In [0]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing import image as kp_image

# Keras is only used to load VGG19 model as a high level API to TensorFlow 
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras import backend as K

# pillow is used for loading and saving images
from PIL import Image

# numPy is used for manipulation of array of object i.e Image in our case
import numpy as np

Using TensorFlow backend.


In [0]:
def load_img(path_to_img):

  max_dim  = 512
  img      = Image.open(path_to_img)
  img = img.convert('RGB')
  img_size = max(img.size)
  scale    = max_dim/img_size
  img      = img.resize((round(img.size[0]*scale), round(img.size[1]*scale)), Image.ANTIALIAS)

  img      = kp_image.img_to_array(img)

  # We need to broadcast the image array such that it has a batch dimension 
  img = np.expand_dims(img, axis=0)

  # preprocess raw images to make it suitable to be used by VGG19 model
  out = tf.keras.applications.vgg19.preprocess_input(img)

  return tf.convert_to_tensor(out)

def deprocess_img(processed_img):
  x = processed_img.copy()
  
  # perform the inverse of the preprocessiing step
  x[:, :, 0] += 103.939
  x[:, :, 1] += 116.779
  x[:, :, 2] += 123.68

  x = x[:, :, ::-1]

  x = np.clip(x, 0, 255).astype('uint8')

  return x

### Build VGG

In [0]:
# list of layers to be considered for calculation of Content and Style Loss
content_layers = ['block3_conv3']
style_layers   = ['block1_conv1','block2_conv2','block4_conv3']

num_content_layers = len(content_layers)
num_style_layers   = len(style_layers)

# path where the content and style images are located
content_path = 'content.jpg'
style_path   = 'style.jpg'

# Save the result as
save_name = 'generated.jpg'

# path to where Vgg19 model weight is located
vgg_weights = "vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"

In [0]:
def get_model(content_layers,style_layers):

  # Load our model. We load pretrained VGG, trained on imagenet data
  vgg19 = VGG19(weights='imagenet', include_top=False)

  # We don't need to (or want to) train any layers of our pre-trained vgg model, so we set it's trainable to false.
  vgg19.trainable = False

  style_model_outputs = [vgg19.get_layer(name).output for name in style_layers]
  content_model_outputs =  [vgg19.get_layer(name).output for name in content_layers]
  
  model_outputs = content_model_outputs + style_model_outputs

  # Build model 
  return Model(inputs = vgg19.input, outputs = model_outputs), vgg19

### Compute Losses

In [0]:
### Content Loss Function
def get_content_loss(content, target):
  return tf.reduce_mean(tf.square(content - target)) /2


### Style Loss Fucntion
def gram_matrix(input_tensor):

  # if input tensor is a 3D array of size Nh x Nw X Nc
  # we reshape it to a 2D array of Nc x (Nh*Nw)
  
  channels = int(input_tensor.shape[-1])
  a = tf.reshape(input_tensor, [-1, channels])
  n = tf.shape(a)[0]

  # get gram matrix 
  gram = tf.matmul(a, a, transpose_a=True)
  
  return gram

def get_style_loss(base_style, gram_target):

  height, width, channels = base_style.get_shape().as_list()
  gram_style = gram_matrix(base_style)
  
  # Original eqn as a constant to divide i.e 1/(4. * (channels ** 2) * (width * height) ** 2)
  return tf.reduce_mean(tf.square(gram_style - gram_target)) / (channels**2 * width * height) #(4.0 * (channels ** 2) * (width * height) ** 2)



### Use to pass content and style image through it 
def get_feature_representations(model, content_path, style_path, num_content_layers):

  # Load our images in 
  content_image = load_img(content_path)
  style_image   = load_img(style_path)
  
  # batch compute content and style features
  content_outputs = model(content_image)
  style_outputs   = model(style_image)
  
  # Get the style and content feature representations from our model  
  style_features   = [ style_layer[0]  for style_layer    in style_outputs[num_content_layers:] ]
  content_features = [ content_layer[0] for content_layer in content_outputs[:num_content_layers] ]

  return style_features, content_features


### Total Loss
def compute_loss(model, loss_weights, generated_output_activations, gram_style_features, content_features, num_content_layers, num_style_layers):

  generated_content_activations = generated_output_activations[:num_content_layers]
  generated_style_activations   = generated_output_activations[num_content_layers:]

  style_weight, content_weight = loss_weights
  
  style_score = 0
  content_score = 0

  # Accumulate style losses from all layers
  # Here, we equally weight each contribution of each loss layer
  weight_per_style_layer = 1.0 / float(num_style_layers)
  for target_style, comb_style in zip(gram_style_features, generated_style_activations):
    temp = get_style_loss(comb_style[0], target_style)
    style_score += weight_per_style_layer * temp
    
  # Accumulate content losses from all layers 
  weight_per_content_layer = 1.0 / float(num_content_layers)
  for target_content, comb_content in zip(content_features, generated_content_activations):
    temp = get_content_loss(comb_content[0], target_content)
    content_score += weight_per_content_layer* temp

  # Get total loss
  loss = style_weight*style_score + content_weight*content_score 


  return loss, style_score, content_score

# TRAINING Procedure

In [0]:
def run_style_transfer(content_path, style_path, num_iterations=250, content_weight=0.1, style_weight=0.9): 

  # Create a tensorflow session 
  sess = tf.Session()

  # Assign keras back-end to the TF session which we created
  K.set_session(sess)

  model, vgg19 = get_model(content_layers,style_layers)

  # Get the style and content feature representations (from our specified intermediate layers) 
  style_features, content_features = get_feature_representations(model, content_path, style_path, num_content_layers)
  gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

  # VGG default normalization
  norm_means = np.array([103.939, 116.779, 123.68])
  min_vals = -norm_means
  max_vals = 255 - norm_means 
    

  # In original paper, the initial stylized image is random matrix of same size as that of content image
  # but in later images content image was used instead on random values for first stylized image
  # because it proved to help to stylize faster
  generated_image = load_img(content_path)
  # generated_image = np.random.randint(0,255, size=generated_image.shape) 
  
  # Create tensorflow variable to hold a stylized/generated image during the training 
  generated_image = tf.Variable(generated_image, dtype=tf.float32)

  model_outputs = model(generated_image)

  # weightages of each content and style images i.e alpha & beta
  loss_weights = (style_weight, content_weight)

  # Create our optimizer
  loss = compute_loss(model, loss_weights, model_outputs, gram_style_features, content_features, num_content_layers, num_style_layers)
  opt = tf.train.AdamOptimizer(learning_rate=9, beta1=0.9, epsilon=1e-1).minimize( loss[0], var_list = [generated_image])
  
  sess.run(tf.global_variables_initializer())
  sess.run(generated_image.initializer)
  # loading the weights again because tf.global_variables_initializer() resets the weights
  vgg19.load_weights(vgg_weights)
  # Put loss as infinity before training starts and Create a variable to hold best image (i.e image with minimum loss)
  best_loss, best_img = float('inf'), None

  for i in range(num_iterations):

    # Do optimization
    sess.run(opt)

    # Make sure image values stays in the range of max-min value of VGG norm 
    clipped = tf.clip_by_value(generated_image, min_vals, max_vals)
    # assign the clipped value to the tensor stylized image
    generated_image.assign(clipped)


    # Open the Tuple of tensors 
    total_loss, style_score, content_score = loss
    total_loss = total_loss.eval(session=sess)


    if total_loss < best_loss:

      # Update best loss and best image from total loss. 
      best_loss = total_loss

      # generated image is of shape (1, h, w, 3) convert it to (h, w, 3)
      temp_generated_image = sess.run(generated_image)[0]
      best_img = deprocess_img(temp_generated_image)

      s_loss = sess.run(style_score)
      c_loss = sess.run(content_score)

      # print best loss
      print('best: iteration: ', i ,'loss: ', total_loss ,'  style_loss: ',  s_loss,'  content_loss: ', c_loss)

    # Save image after every 100 iterations 
    #if (i+1)%100 == 0:
      #output = Image.fromarray(best_img)
      #output.save(str(i+1)+'-'+save_name)

  # after num_iterations iterations are completed, close the TF session 
  sess.close()
      
  return best_img, best_loss, best_img

In [0]:
#!wget https://github.com/hwang123/865_final_project/raw/master/dan_yaha.jpg
#!wget https://github.com/hwang123/865_final_project/raw/master/mondrian.jpg
#!wget https://github.com/Shashi456/Neural-Style/raw/master/Neural%20Style%20Transfer/examples/content/dancing.jpg
!wget https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
best, best_loss, img = run_style_transfer('dan_yaha.jpg', 'mondrian.jpg', content_weight = 0.95, style_weight = 0.05)
img = Image.fromarray(img)
img.show()

In [0]:
img.save('/content/TESTDANWEIGHTS.png')

In [0]:
best, best_loss, img = run_style_transfer('dan_yaha.jpg', '/content/sketch.jpg', content_weight = 0.2, style_weight = 0.8)
img = Image.fromarray(img)
img.show()
img.save('/content/TESTDANSKETCH.png')

In [0]:
best, best_loss, img = run_style_transfer('dan_yaha.jpg', '/content/cartoon.jpeg', content_weight = 0.2, style_weight = 0.8)
img = Image.fromarray(img)
img.show()
img.save('/content/TESTDANCARTOON.png')

In [0]:
!wget https://github.com/hwang123/865_final_project/raw/master/content/dog0.png

In [0]:
best, best_loss, img = run_style_transfer('dog0.png', '/content/sketch.jpg', content_weight = 0.2, style_weight = 0.8)
img = Image.fromarray(img)
img.show()
img.save('/content/TESTDOG.png')

In [0]:
best, best_loss, img = run_style_transfer('zebra.jpg', '/content/pencil.jpg', num_iterations=100, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.show()
img.save('/content/TESTZEBRAPENCIL.png')

In [0]:
best, best_loss, img = run_style_transfer('zebra0.png', '/content/lisafrank.jpg', content_weight = 0.01, style_weight = 0.99)
img = Image.fromarray(img)
img.show()
img.save('/content/TESTZEBRARAINBOWNEW.png')

In [0]:
!wget https://github.com/hwang123/865_final_project/raw/master/content/zebra0.png

In [0]:
best, best_loss, img = run_style_transfer('zebra0.png', '/content/linedrawing.png', num_iterations = 150, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.show()
img.save('/content/ZEBRALINE.png')

best, best_loss, img = run_style_transfer('zebra0.png', '/content/pop_art.jpeg', num_iterations = 150, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.show()
img.save('/content/ZEBRAPOP.png')

best, best_loss, img = run_style_transfer('zebra0.png', '/content/starry_night.jpeg', num_iterations = 150, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.show()
img.save('/content/ZEBRAVANGOGH.png')


In [0]:
best, best_loss, img = run_style_transfer('/content/car2.png', '/content/sketch.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carsketch2.jpeg')

best, best_loss, img = run_style_transfer('/content/car2.png', '/content/lisafrank.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carlisafrank2.jpeg')

best, best_loss, img = run_style_transfer('/content/car2.png', '/content/pop_art.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carpopart2.jpeg')

best, best_loss, img = run_style_transfer('/content/car2.png', '/content/starry_night.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carstarrynight2.jpeg')

best, best_loss, img = run_style_transfer('/content/car2.png', '/content/linedrawing.png', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carlinedrawing2.jpeg')

best, best_loss, img = run_style_transfer('/content/car2.png', '/content/pencil.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/carpencil2.jpeg')

Instructions for updating:
Colocations handled automatically by placer.
80142336/80134624 [==============================] - 2s 0us/step
best: iteration:  0 loss:  662301440.0   style_loss:  735888900.0   content_loss:  13955.267
best: iteration:  1 loss:  488442200.0   style_loss:  542709900.0   content_loss:  33135.402
best: iteration:  2 loss:  339263600.0   style_loss:  376953470.0   content_loss:  54899.25
best: iteration:  3 loss:  241033420.0   style_loss:  267806820.0   content_loss:  73027.234
best: iteration:  4 loss:  197597520.0   style_loss:  219543260.0   content_loss:  85861.66
best: iteration:  5 loss:  188532080.0   style_loss:  209469820.0   content_loss:  92356.516
best: iteration:  6 loss:  180023090.0   style_loss:  200015330.0   content_loss:  93150.78
best: iteration:  7 loss:  160423890.0   style_loss:  178238700.0   content_loss:  90613.695
best: iteration:  8 loss:  138895410.0   style_loss:  154318530.0   content_loss:  87441.55
best: iteration:  9 loss:  122

In [0]:
best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/sketch.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickupsketch2.jpeg')

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/sketch.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/trucksketch2.jpeg')

best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/lisafrank.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickuplisafrank2.jpeg')

best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/pop_art.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickuppopart2.jpeg')

best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/starry_night.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickupstarrynight2.jpeg')

best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/linedrawing.png', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickuplinedrawing2.jpeg')

best, best_loss, img = run_style_transfer('/content/pickup2.png', '/content/pencil.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/pickuppencil2.jpeg')

In [0]:

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/lisafrank.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/trucklisafrank2.jpeg')

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/pop_art.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/truckpopart2.jpeg')

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/starry_night.jpeg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/truckstarrynight2.jpeg')

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/linedrawing.png', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/trucklinedrawing2.jpeg')

best, best_loss, img = run_style_transfer('/content/truck2.png', '/content/pencil.jpg', num_iterations = 200, content_weight = 0.1, style_weight = 0.9)
img = Image.fromarray(img)
img.save('/content/truckpencil2.jpeg')

best: iteration:  0 loss:  1967006800.0   style_loss:  2185561000.0   content_loss:  19133.48
best: iteration:  1 loss:  1093010700.0   style_loss:  1214450300.0   content_loss:  54681.74
best: iteration:  2 loss:  401212930.0   style_loss:  445780350.0   content_loss:  106296.47
best: iteration:  6 loss:  313963040.0   style_loss:  348830140.0   content_loss:  159158.75
best: iteration:  7 loss:  156198060.0   style_loss:  173537870.0   content_loss:  139875.06
best: iteration:  8 loss:  152772200.0   style_loss:  169733000.0   content_loss:  124933.17
best: iteration:  13 loss:  124444860.0   style_loss:  138254770.0   content_loss:  155723.31
best: iteration:  17 loss:  113405370.0   style_loss:  125986580.0   content_loss:  174541.72
best: iteration:  18 loss:  84079920.0   style_loss:  93403720.0   content_loss:  165781.33
best: iteration:  19 loss:  72934040.0   style_loss:  81020360.0   content_loss:  157221.89
best: iteration:  23 loss:  69114790.0   style_loss:  76777290.0   c

# New Section